# 11. GEOSPATIAL ANALYSIS

# 11.4. Manipulating Geospatiale Data

# 11.4.1. COURS

# Geocoding

In [1]:
!pip install geopy==1.22.0
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker

In [2]:
from geopandas.tools import geocode

In [3]:
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

,geometry,address
0,POINT (31.13422 29.97916),"هرم خوفو, Cause way, كوم الأخضر, الجيزة, محافظ..."


In [4]:
point = result.geometry.iloc[0]
print("Latitude:", point.y)
print("Longitude:", point.x)

Latitude: 29.97915995
Longitude: 31.134215650388754


In [5]:
universities = pd.read_csv("geospatial-learn-course-data/input/top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [6]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

universities[['Latitude', 'Longitude', 'geometry']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(universities, geometry=universities.geometry)
universities.crs = {'init': 'epsg:4326'}
universities.head()

90.0% of addresses were geocoded!


C:\Users\Outils\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871)
1,University of Cambridge,52.199852,0.119739,POINT (0.11974 52.19985)
2,Imperial College London,51.498983,-0.174629,POINT (-0.17463 51.49898)
3,ETH Zurich,47.377327,8.547509,POINT (8.54751 47.37733)
4,UCL,51.523564,-0.132956,POINT (-0.13296 51.52356)


In [7]:
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
m

# Table joins

# Attribute join

In [8]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
europe = world.loc[world.continent == 'Europe'].reset_index(drop=True)

europe_stats = europe[["name", "pop_est", "gdp_md_est"]]
europe_boundaries = europe[["name", "geometry"]]

In [9]:
europe_boundaries.head()

,name,geometry
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516..."
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ..."
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ..."
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300..."
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372..."


In [10]:
europe_stats.head()

,name,pop_est,gdp_md_est
0,Russia,142257519,3745000.0
1,Norway,5320045,364700.0
2,France,67106161,2699000.0
3,Sweden,9960487,498100.0
4,Belarus,9549747,165400.0


In [11]:
# Use an attribute join to merge data about countries in Europe
europe = europe_boundaries.merge(europe_stats, on="name")
europe.head()

,name,geometry,pop_est,gdp_md_est
0,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516...",142257519,3745000.0
1,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ...",5320045,364700.0
2,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ...",67106161,2699000.0
3,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300...",9960487,498100.0
4,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372...",9549747,165400.0


# Spatial join

In [12]:
# Use spatial join to match universities to countries in Europe
european_universities = gpd.sjoin(universities, europe)

# Investigate the result
print("We located {} universities.".format(len(universities)))
print("Only {} of the universities were located in Europe (in {} different countries).".format(
    len(european_universities), len(european_universities.name.unique())))

european_universities.head()

We located 90 universities.
Only 86 of the universities were located in Europe (in 15 different countries).


<ipython-input-12-146764234e5d>:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  european_universities = gpd.sjoin(universities, europe)


,Name,Latitude,Longitude,geometry,index_right,name,pop_est,gdp_md_est
0,University of Oxford,51.758708,-1.255668,POINT (-1.25567 51.75871),28,United Kingdom,64769452,2788000.0
1,University of Cambridge,52.199852,0.119739,POINT (0.11974 52.19985),28,United Kingdom,64769452,2788000.0
2,Imperial College London,51.498983,-0.174629,POINT (-0.17463 51.49898),28,United Kingdom,64769452,2788000.0
4,UCL,51.523564,-0.132956,POINT (-0.13296 51.52356),28,United Kingdom,64769452,2788000.0
5,London School of Economics and Political Science,51.514591,-0.116431,POINT (-0.11643 51.51459),28,United Kingdom,64769452,2788000.0


# 11.4.2. EXERCICES

In [13]:
import math
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode           
import folium 
from folium import Marker
from folium.plugins import MarkerCluster


In [14]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# 1) Geocode the missing locations

In [15]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("geospatial-learn-course-data/input/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


In [16]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


In [17]:
# Your code here
def my_geocoder(row):
    point = geocode(row, provider='nominatim').geometry[0]
    return pd.Series({'Longitude': point.x, 'Latitude': point.y})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

# 2) View Berkeley locations.

In [18]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Your code here: Add a marker for each Berkeley location
# Add points to the map
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)

# Show the map
embed_map(m_2, 'q_2.html')

# 3) Consolidate your data.

In [20]:
CA_counties = gpd.read_file("geospatial-learn-course-data/input/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


In [21]:
CA_pop = pd.read_csv("geospatial-learn-course-data/input/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("geospatial-learn-course-data/input/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("geospatial-learn-course-data/input/CA_county_median_age.csv", index_col="GEOID")

In [25]:
# Your code here
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")
CA_stats

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0
6,6097,Sonoma County,4578.952090,"POLYGON ((-122.93507 38.31396, -122.93511 38.3...",499942,32713,41.4
7,6031,Kings County,3604.052342,"POLYGON ((-119.95894 36.25547, -119.95894 36.2...",151366,2943,31.5
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4
9,6061,Placer County,3890.821444,"POLYGON ((-121.06545 39.00654, -121.06538 39.0...",393149,28334,41.6


In [28]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

In [31]:
sel_counties = CA_stats[((CA_stats.high_earners >= 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners >= 500000)))]
sel_counties

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


# 5) How many stores did you identify?

In [32]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

C:\Users\Outils\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [34]:
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
locations_of_interest 

<ipython-input-34-3c62361b131c>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)


,Store Number,Store Name,Address,City,Longitude,Latitude,geometry,index_right,GEOID,name,area_sqkm,population,high_earners,median_age,density
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
14,76365-97162,Target Alhambra T-184,1220 West Main Street Alhambra CA,Alhambra,-118.14,34.09,POINT (-118.14000 34.09000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
15,6794-41839,Fremont Ave & Mission Rd,"1131 S Fremont Ave, A Alhambra CA",Alhambra,-118.15,34.08,POINT (-118.15000 34.08000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
16,11220-104633,"Atlantic & Valley, Alhambra",1410 South Atlantic Blvd. Alhambra CA,Alhambra,-118.13,34.08,POINT (-118.13000 34.08000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2138,5836-13469,3rd & Howard,201 3rd St. San Francisco CA,San Francisco,-122.40,37.78,POINT (-122.40000 37.78000),10,6075,San Francisco County,600.588247,883305,114989,38.3,1470.733077
2139,545-488,24th & Noe,3995 24th Street San Francisco CA,San Francisco,-122.43,37.75,POINT (-122.43000 37.75000),10,6075,San Francisco County,600.588247,883305,114989,38.3,1470.733077
2140,14311-115765,Owens & 16th,"1700 Owens Street, Third & Market San Francisc...",San Francisco,-122.39,37.77,POINT (-122.39000 37.77000),10,6075,San Francisco County,600.588247,883305,114989,38.3,1470.733077
2141,10347-100168,Third & Market,7 Third Street San Francisco CA,San Francisco,-122.40,37.79,POINT (-122.40000 37.79000),10,6075,San Francisco County,600.588247,883305,114989,38.3,1470.733077


In [37]:
num_stores = len(locations_of_interest)
num_stores

1043

# 6) Visualize the store locations.

In [38]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Your code here: show selected store locations
# Show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)

# Show the map
embed_map(m_6, 'q_6.html')

<ipython-input-38-ff3ca9cc82fe>:8: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:4326 +type=crs
Right CRS: EPSG:4326

  locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
